In [41]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV

In [42]:
# load data
data = pd.read_csv("../data/final_merged_data.csv")
data = data.iloc[:, 1:] # drop first index column

In [43]:
# Configuration
window_size = 172  # Number of months to use for training
oos_months = 1    # Number of months to predict
total_months = 272  # Total number of months (for example, 2 years of data)
num_stocks = 193   # Total number of stocks

# Load characteristics and returns data
c = data.drop(columns=['Date', "Risk_Premium"])  # Characteristics data (shape: [total_months * num_stocks, num_features]), num_features = 116
r1 = data[['Risk_Premium']]  # Returns data (shape: [total_months * num_stocks])

In [44]:
# one-hot encoding ticker (categoritcal variable to dummy)
c_encoded = pd.get_dummies(c, drop_first=True)  # Drop first to avoid multicollinearity
c_encoded

,Open_lag_1,High_lag_1,Low_lag_1,Close_lag_1,Volume_lag_1,Dividends_lag_1,Stock Splits_lag_1,Monthly_Return_lag_1,3_Month_Treasury_Bill_Yield_lag_1,6_Month_Treasury_Bill_Yield_lag_1,...,Ticker_VOXX,Ticker_VRSN,Ticker_VSAT,Ticker_VSH,Ticker_VYX,Ticker_WDC,Ticker_WOLF,Ticker_WYY,Ticker_XRX,Ticker_ZBRA
0,0.271611,0.354930,0.252311,0.346457,1.588897e+10,0.0,0.0,0.279997,0.0429,0.0437,...,False,False,False,False,False,False,False,False,False,False
1,0.346927,0.376584,0.308327,0.331864,8.908906e+09,0.0,0.0,-0.042120,0.0450,0.0456,...,False,False,False,False,False,False,False,False,False,False
2,0.338925,0.369052,0.316801,0.348810,6.918621e+09,0.0,0.0,0.051063,0.0457,0.0482,...,False,False,False,False,False,False,False,False,False,False
3,0.348810,0.426481,0.340338,0.419420,1.146658e+10,0.0,0.0,0.202431,0.0455,0.0458,...,False,False,False,False,False,False,False,False,False,False
4,0.418949,0.496149,0.392589,0.491441,9.244603e+09,0.0,0.0,0.171716,0.0472,0.0487,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52491,530.000000,554.219971,502.630005,552.479980,4.412900e+06,0.0,0.0,0.043419,0.0005,0.0005,...,False,False,False,False,False,False,False,False,False,True
52492,555.000000,594.140015,527.539978,587.169983,4.195600e+06,0.0,0.0,0.062790,0.0005,0.0006,...,False,False,False,False,False,False,False,False,False,True
52493,592.479980,594.770020,514.989990,515.419983,5.111900e+06,0.0,0.0,-0.122196,0.0004,0.0005,...,False,False,False,False,False,False,False,False,False,True
52494,517.619995,543.489990,491.989990,533.950012,4.694900e+06,0.0,0.0,0.035951,0.0005,0.0006,...,False,False,False,False,False,False,False,False,False,True


In [45]:
# Check for columns with missing values
missing_columns = c_encoded.isna().any()

print("Columns with missing values:")
print(missing_columns[missing_columns]) 

c_encoded[c_encoded['mvel1'].isna()]

Columns with missing values:
mvel1         True
beta          True
betasq        True
chmom         True
dolvol        True
              ... 
retvol        True
std_dolvol    True
std_turn      True
zerotrade     True
sic2          True
Length: 95, dtype: bool


,Open_lag_1,High_lag_1,Low_lag_1,Close_lag_1,Volume_lag_1,Dividends_lag_1,Stock Splits_lag_1,Monthly_Return_lag_1,3_Month_Treasury_Bill_Yield_lag_1,6_Month_Treasury_Bill_Yield_lag_1,...,Ticker_VOXX,Ticker_VRSN,Ticker_VSAT,Ticker_VSH,Ticker_VYX,Ticker_WDC,Ticker_WOLF,Ticker_WYY,Ticker_XRX,Ticker_ZBRA
5984,51.000000,81.000,43.125000,76.500000,3533264.0,0.0,0.0,0.522388,0.0429,0.0437,...,False,False,False,False,False,False,False,False,False,False
5985,76.875000,91.125,67.875000,75.750000,2281500.0,0.0,0.0,-0.009804,0.0450,0.0456,...,False,False,False,False,False,False,False,False,False,False
5986,76.125000,91.500,63.750000,76.500000,3283050.0,0.0,0.0,0.009901,0.0457,0.0482,...,False,False,False,False,False,False,False,False,False,False
5987,76.125000,91.500,68.625000,71.250000,2317032.0,0.0,0.0,-0.068627,0.0455,0.0458,...,False,False,False,False,False,False,False,False,False,False
5988,70.875000,90.000,63.000000,82.125000,2424733.0,0.0,0.0,0.152632,0.0472,0.0487,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51764,25.500000,31.500,25.200001,29.500000,699360.0,0.0,0.0,0.130268,0.0460,0.0472,...,False,False,False,False,False,False,False,True,False,False
51765,29.799999,30.500,26.500000,28.200001,716170.0,0.0,0.0,-0.044068,0.0472,0.0482,...,False,False,False,False,False,False,False,True,False,False
51766,28.000000,30.000,25.000000,28.600000,372500.0,0.0,0.0,0.014184,0.0479,0.0497,...,False,False,False,False,False,False,False,True,False,False
51767,29.700001,30.500,27.299999,28.000000,401900.0,0.0,0.0,-0.020979,0.0495,0.0506,...,False,False,False,False,False,False,False,True,False,False


In [46]:
# Add a column indicating if any row has NA and impute missing values
c_encoded['has_na'] = c_encoded.isna().any(axis=1)

# Impute missing values with the mean for numerical columns
imputer = SimpleImputer(strategy='mean')
c_encoded.iloc[:, :-1] = imputer.fit_transform(c_encoded.iloc[:, :-1])
c_encoded

C:\Users\yangt\AppData\Local\Temp\ipykernel_46740\310738126.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0. 0. 0. ... 0. 0. 0.]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  c_encoded.iloc[:, :-1] = imputer.fit_transform(c_encoded.iloc[:, :-1])
C:\Users\yangt\AppData\Local\Temp\ipykernel_46740\310738126.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0. 0. 0. ... 0. 0. 0.]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  c_encoded.iloc[:, :-1] = imputer.fit_transform(c_encoded.iloc[:, :-1])
C:\Users\yangt\AppData\Local\Temp\ipykernel_46740\310738126.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0. 0. 0. ... 0. 0. 0.]' has dtype incompatible with bool, please explicitly cas

,Open_lag_1,High_lag_1,Low_lag_1,Close_lag_1,Volume_lag_1,Dividends_lag_1,Stock Splits_lag_1,Monthly_Return_lag_1,3_Month_Treasury_Bill_Yield_lag_1,6_Month_Treasury_Bill_Yield_lag_1,...,Ticker_VRSN,Ticker_VSAT,Ticker_VSH,Ticker_VYX,Ticker_WDC,Ticker_WOLF,Ticker_WYY,Ticker_XRX,Ticker_ZBRA,has_na
0,0.271611,0.354930,0.252311,0.346457,1.588897e+10,0.0,0.0,0.279997,0.0429,0.0437,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
1,0.346927,0.376584,0.308327,0.331864,8.908906e+09,0.0,0.0,-0.042120,0.0450,0.0456,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
2,0.338925,0.369052,0.316801,0.348810,6.918621e+09,0.0,0.0,0.051063,0.0457,0.0482,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
3,0.348810,0.426481,0.340338,0.419420,1.146658e+10,0.0,0.0,0.202431,0.0455,0.0458,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
4,0.418949,0.496149,0.392589,0.491441,9.244603e+09,0.0,0.0,0.171716,0.0472,0.0487,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52491,530.000000,554.219971,502.630005,552.479980,4.412900e+06,0.0,0.0,0.043419,0.0005,0.0005,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,False
52492,555.000000,594.140015,527.539978,587.169983,4.195600e+06,0.0,0.0,0.062790,0.0005,0.0006,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,False
52493,592.479980,594.770020,514.989990,515.419983,5.111900e+06,0.0,0.0,-0.122196,0.0004,0.0005,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,False
52494,517.619995,543.489990,491.989990,533.950012,4.694900e+06,0.0,0.0,0.035951,0.0005,0.0006,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,False


In [24]:
# Store results
results = []

# Rolling window approach
for start in range(total_months - window_size):
    # Define training and OOS sets
    xtrain = c_encoded[start * num_stocks:(start + window_size) * num_stocks]
    ytrain = r1[start * num_stocks:(start + window_size) * num_stocks]
    xoos = c_encoded[(start + window_size) * num_stocks:(start + window_size + oos_months) * num_stocks]
    yoos = r1[(start + window_size) * num_stocks:(start + window_size + oos_months) * num_stocks]

    # Create and train the Gradient Boosted Regression Tree model
    model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42) # 117/3 = 39
    model.fit(xtrain, ytrain.ravel())

    # Predictions for out-of-sample data
    yhat_oos = model.predict(xoos)

    # Ensure yhat_oos is a 1D array
    yhat_oos = np.array(yhat_oos).flatten()  # This ensures it is 1D

    # Ensure yoos is a 1D array
    yoos = np.array(yoos).flatten()  # Flattening to convert to 1D

    # Ensure ytrain is a 1D array
    ytrain_flat = np.array(ytrain).flatten()  # Flattening ytrain

    # Calculate R-squared for out-of-sample dataset
    r_squared_oos = 1 - np.sum(np.power(yhat_oos - yoos, 2)) / np.sum(np.power(yoos - np.mean(ytrain_flat), 2))

    # Store results, including predicted returns
    for i in range(len(yhat_oos)):
        results.append({
            'start_month': start,
            'R_squared_oos': r_squared_oos,
            'predicted_return': yhat_oos[i],
            'actual_return': yoos[i]
        })

# Convert results to DataFrame for easier analysis
results_df = pd.DataFrame(results)
print(results_df)

# Optionally save results
# results_df.to_csv('rolling_window_results.csv', index=False)

c:\Users\yangt\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\ensemble\_gb.py:668: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)  # TODO: Is this still required?
c:\Users\yangt\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\ensemble\_gb.py:668: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)  # TODO: Is this still required?
c:\Users\yangt\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\ensemble\_gb.py:668: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)  # TODO: Is this still required?
c:\Users\yangt\AppData\Local\Pr

       start_month  R_squared_oos  predicted_return  actual_return
0                0       0.173556         -0.105879      -0.053862
1                0       0.173556          0.014833       0.202260
2                0       0.173556         -0.092535      -0.134121
3                0       0.173556          0.038651      -0.060900
4                0       0.173556         -0.140074      -0.495583
...            ...            ...               ...            ...
19295           99       0.208655          0.006167       0.062290
19296           99       0.208655          0.003965      -0.122596
19297           99       0.208655          0.021483       0.035451
19298           99       0.208655          0.006167       0.102188
19299           99       0.208655          0.007714       0.010304

[19300 rows x 4 columns]


In [25]:
results_df = pd.DataFrame(results)
results_df

,start_month,R_squared_oos,predicted_return,actual_return
0,0,0.173556,-0.105879,-0.053862
1,0,0.173556,0.014833,0.202260
2,0,0.173556,-0.092535,-0.134121
3,0,0.173556,0.038651,-0.060900
4,0,0.173556,-0.140074,-0.495583
...,...,...,...,...
19295,99,0.208655,0.006167,0.062290
19296,99,0.208655,0.003965,-0.122596
19297,99,0.208655,0.021483,0.035451
19298,99,0.208655,0.006167,0.102188


In [26]:
results_df.to_csv('../data/gbrt_results.csv', index=True)

In [39]:
# get overall OOS R-squared
results_df = pd.read_csv("../data/model_results/gbrt_results.csv")
results_df["R_squared_oos"].mean()

0.09284044604389875

In [27]:
# Define the fixed investment amount
fixed_investment = 1000  # Amount to invest each month

# Create a DataFrame for cumulative returns
investment_results = []  # To store cumulative returns
cumulative_return = 0.0  # Start with zero cumulative return

# Loop through each unique month in the results
for month in sorted(results_df['start_month'].unique()):
    # Select the stocks for the current month
    current_month_stocks = results_df[results_df['start_month'] == month]

    if not current_month_stocks.empty:
        # Select the top 10% stocks based on predicted returns
        top_10_percent_threshold = current_month_stocks['predicted_return'].quantile(0.90)
        current_top_stocks = current_month_stocks[current_month_stocks['predicted_return'] >= top_10_percent_threshold]

        # Use the risk premium as the monthly return
        monthly_returns = current_top_stocks['actual_return'].values
        
        # Calculate the average risk premium of the selected top stocks
        if len(monthly_returns) > 0:
            average_monthly_return = monthly_returns.mean()  # Average risk premium
            
            # Calculate total return from the fixed investment
            cumulative_return += average_monthly_return * fixed_investment

            # Store the result for this month
            investment_results.append({
                'month': month,
                'cumulative_return': cumulative_return
            })

# Convert results to DataFrame
investment_df = pd.DataFrame(investment_results)

# Display cumulative returns
print(investment_df)

    month  cumulative_return
0       0         125.039609
1       1         286.934985
2       2         623.811504
3       3         577.928790
4       4         857.516821
..    ...                ...
95     95       11678.461342
96     96       11822.677813
97     97       11954.037014
98     98       11979.929910
99     99       12073.148627

[100 rows x 2 columns]


In [ ]:
# Store results
results = []

# Define hyperparameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [2, 3, 4]
}

# Rolling window approach
for start in range(total_months - window_size):
    # Define training and OOS sets
    xtrain = c_encoded[start * num_stocks:(start + window_size) * num_stocks]
    ytrain = r1[start * num_stocks:(start + window_size) * num_stocks]
    xoos = c_encoded[(start + window_size) * num_stocks:(start + window_size + oos_months) * num_stocks]
    yoos = r1[(start + window_size) * num_stocks:(start + window_size + oos_months) * num_stocks]

    # Ensure ytrain is a 1D array
    ytrain_flat = np.array(ytrain).flatten()
    
    # Perform cross-validation to find the best hyperparameters
    model = GradientBoostingRegressor(random_state=42)
    grid_search = GridSearchCV(model, param_grid, cv=5, scoring='r2', n_jobs=-1)
    grid_search.fit(xtrain, ytrain_flat)
    best_model = grid_search.best_estimator_
    
    # Predictions for out-of-sample data
    yhat_oos = best_model.predict(xoos)
    yhat_oos = np.array(yhat_oos).flatten()
    yoos = np.array(yoos).flatten()

    # Calculate R-squared for out-of-sample dataset
    r_squared_oos = 1 - np.sum(np.power(yhat_oos - yoos, 2)) / np.sum(np.power(yoos - np.mean(ytrain_flat), 2))

    # Store results, including predicted returns
    for i in range(len(yhat_oos)):
        results.append({
            'start_month': start,
            'R_squared_oos': r_squared_oos,
            'predicted_return': yhat_oos[i],
            'actual_return': yoos[i],
            'best_params': grid_search.best_params_
        })

# Convert results to DataFrame for easier analysis
results_df = pd.DataFrame(results)
print(results_df)